In [12]:
#!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon_10_0.899.h5

# convert model to TFlite

In [5]:
import tensorflow as tf
from tensorflow import keras

In [8]:
model = keras.models.load_model('dino_dragon_10_0.899.h5')

2022-12-20 20:07:08.913849: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-12-20 20:07:08.917537: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-20 20:07:08.934718: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('dragon_model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

2022-12-20 20:08:00.828488: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmphagbuibk/assets


2022-12-20 20:08:01.391831: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-12-20 20:08:01.392123: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-12-20 20:08:01.393125: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1800000000 Hz
2022-12-20 20:08:01.409727: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:928] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 1.579ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2022-12-20 20:08:02.407774: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:316] Ignored output_format.
2022-12-20 20:08:02.407877: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:319] Ignored drop_control_dependency.
2022-12-20 20:08:02.674868

In [11]:
ls -la

total 131480
drwxrwxrwx 1 rollytiger rollytiger      512 Dec 20 20:08 ./
drwxrwxrwx 1 rollytiger rollytiger      512 Dec 20 20:03 ../
drwxrwxrwx 1 rollytiger rollytiger      512 Dec 20 20:05 .ipynb_checkpoints/
-rwxrwxrwx 1 rollytiger rollytiger 89759168 Nov 19 19:23 dino_dragon_10_0.899.h5*
-rwxrwxrwx 1 rollytiger rollytiger 44865888 Dec 20 20:08 dragon_model.tflite*
-rwxrwxrwx 1 rollytiger rollytiger     6369 Dec 20 20:08 hw9_notebook.ipynb*


# use the tflite model

In [1]:
import tflite_runtime.interpreter as tflite

In [2]:
interpreter = tflite.Interpreter(model_path='dragon_model.tflite')
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


## load image

In [3]:
import numpy as np
from PIL import Image
from urllib import request
from io import BytesIO
   
def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

def preprocess_manual(x):
    x /= 255   
    return x

classes = ['dino',
           'dragon']

In [4]:
url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg'

In [28]:
def predict(url):
    
    response = request.urlopen(url)
    with Image.open(BytesIO(response.read())) as img:
        img = img.resize((150,150), Image.NEAREST)
        
    x = np.array(img, dtype = 'float32')
    X = np.array([x])
    X = preprocess_manual(X)
    
    interpreter.set_tensor(input_index, X)
    interpreter.invoke()
    
    tf_preds = interpreter.get_tensor(output_index)
    
    dicta = {}

    if tf_preds[0][0] > 0.5:
        animal = 'dragon'
    else:
        animal = 'dino'

    dicta[animal] = str(tf_preds[0][0])
    
    return dicta

In [29]:
predict(url)

/tmp/ipykernel_289/1966568172.py:5: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.resize((150,150), Image.NEAREST)


{'dragon': '0.82448506'}